# Underwater Glider Data - An intuitive data structure

How to create a hierarchical data structure for underwater glider measurements. This example will consider measurements from an underwater glider, but the same principle could be applied to different observing platforms, such as Argo, Saildrone, wave gliders, and even shipboard CTDs.

## Objectives

- Demonstrate xarray-datatree on observing data, in contrast to regularly gridded data such as from numerical models or satellite images.
- Walk through the process of creating a netCDF-CF dataset organized with groups.
- Use simplified but realistic metadata, measurements, and structure.

## Intro

### Dataset characteristics

In this tutorial, we will assume measurements from an underwater glider, which are autonomous platforms, i.e. robots, that can dive and return to the surface by controlling its buoyancy. These are similar to Argo floats but equipped with wings and a rudder, allowing them to control a relatively slow horizontal displacement. A typical operation hence results in measurements from different instruments along a sawtooth-like path. While a typical setup includes a GPS (geolocation) and a CTD (pressure, temperature, and conductivity, thus salinity), a glider can be also equipped with a diverse set of instruments such as ADCP (velocity structure), fluorometer (chlorophyll, CDOM, …) and oximeter, to name a few. All these instruments don’t necessarily operate at the same time or even with the same sampling frequency, making it sometimes awkward to cluster those measurements together. A more intuitive structure would share the platform, thus the same track, but allow for each instrument freedom for its natural structure and metadata.

A proper self-describing dataset requires sufficient metadata which can make it cumbersome to understand. On top of that, underwater gliders are autonomous platforms equipped with several sensors that don't necessarily operate synchronously, hence a mission (equivalent to a cruise for ships) shall be the composition of multiple sub-datasets with different dimensions. A natural solution to organize it without losing information is by using groups, a structure available on netCDF-4 and defined on the CF-Conventions. This structure is currently been proposed for the OceanGlidersCommunity data format, `OG-1.0`.

Here we will simulate how to produce a netCDF for an underwater glider organized in groups. Other types of observing platforms could follow the same principle, such as BGC-Argo. An hydrographic cruise is probably the most clear  example of a consistent dataset of several independent instruments.

In [1]:
# Let's import everything that we will need

import numpy as np
import xarray as xr
from datatree import DataTree

Let's start by creating the main object that will represent one deployment, and call it a `mission`.

In [2]:
mission = DataTree()

## Global Attributes

This deployment has some top level metadata related to the full deployment. For instance, the `Conventions` that will be used on this dataset, a `title`, and a `doi` to be cited when using it.

In [3]:
mission.attrs["Conventions"] = "CF-1.9, ACDD-1.3, OG-1.0rc"
mission.attrs["title"] = "Spray Underwater glider demo"
mission.attrs["platform"] = "sp041"
mission.attrs["project"] = "California Underwater Glider Network"
mission.attrs["doi"] = "10.21238/S8SPRAY1618"
mission.attrs["comment"] = "Dataset for demonstration purposes only. Original dataset truncated for the sake of simplicity"

print(mission)

DataTree('None', parent=None)
    Dimensions:  ()
    Data variables:
        *empty*
    Attributes:
        Conventions:  CF-1.9, ACDD-1.3, OG-1.0rc
        title:        Spray Underwater glider demo
        platform:     sp041
        project:      California Underwater Glider Network
        doi:          10.21238/S8SPRAY1618
        comment:      Dataset for demonstration purposes only. Original dataset t...


## Sensors

Let's move to the sensors. A glider usually is equipped with GPS and CTD, but there is a wide variety of possible payloads including fluorometer, oximeter, ADCP, and others.

### CTD

Let's assume that we received some real-time data through Iridium. To keep it simple, let's show only the end of a dive.

In [4]:
time = np.array(['2019-12-16T17:22:41', '2019-12-16T17:22:49', '2019-12-16T17:22:57', '2019-12-16T17:23:05', '2019-12-16T17:23:13'], dtype='datetime64')
pressure = np.array([5.32, 4.36, 3.4 , 2.32, 1.56])
temperature = np.array([15.511, 15.51 , 15.511, 15.512, 15.512])
salinity = np.array([33.561, 33.56 , 33.56 , 33.559, 33.56 ])

The first step would be to describe its structure and augment with some metadata. Let's use `xarray` for that,

In [5]:
ds_ctd = xr.Dataset(dict(
    time=(["time"], time, {
        "long_name": "Time",
        "standard_name": "time",
    }),
    press=(["time"], pressure, {
        'long_name': 'Pressure',
        'standard_name': 'sea_water_pressure',
        'units': 'dbar',
        'comment': 'Sea water pressure, equals 0 at sea-level'}),
    temp=(["time"], temperature, {
        'long_name': 'Sea Water Temperature',
        'standard_name': 'sea_water_temperature',
        'units': 'Celsius',
        'comment': 'Sea temperature in-situ ITS-90 scale'}),
    sal=(["time"], salinity, {
        'long_name': 'Sea Water Salinity',
        'standard_name': 'sea_water_practical_salinity',
        'units': '1',
        'comment': 'Practical salinity computed using UNESCO 1983 algorithm'}),
))

print(ds_ctd)

<xarray.Dataset>
Dimensions:  (time: 5)
Coordinates:
  * time     (time) datetime64[ns] 2019-12-16T17:22:41 ... 2019-12-16T17:23:13
Data variables:
    press    (time) float64 5.32 4.36 3.4 2.32 1.56
    temp     (time) float64 15.51 15.51 15.51 15.51 15.51
    sal      (time) float64 33.56 33.56 33.56 33.56 33.56


Now we have a consistent structure, where each measurement is clearly associated with a time.

For the final product, we wouldn't need to define the `calendar` or `units` for time, but if we wanted to force it to create consistency we would need to define the encoding of the `time`.

In [6]:
ds_ctd["time"].encoding["calendar"] = "gregorian"
ds_ctd["time"].encoding["units"] = "seconds since 1970-01-01T00:00:00Z"

Now let's add this CTD dataset to our mission dataset,

In [7]:
ctd = DataTree(data=ds_ctd, name="CTD", parent=mission)

We also have specific metadata for the CTD, such as its specification, serial number, calibration date, ...

In [8]:
ctd.attrs["long_name"] = "CTD sensor"
ctd.attrs["type"] = "CTD"
ctd.attrs["type_identifier"] = "https://vocab.nerc.ac.uk/collection/L05/current/130/"
ctd.attrs["maker"] = "Sea-Bird Scientific"
ctd.attrs["maker_identifier"] = "https://vocab.nerc.ac.uk/collection/L35/current/MAN0013/"
ctd.attrs["model"] = "SBE 41CP CTD "
ctd.attrs["model_identifier"] = "https://vocab.nerc.ac.uk/collection/L22/current/TOOL0669/"
ctd.attrs["serial_number"] = "75"
ctd.attrs["calibration_date"] = "2019-10-22T00:00:00Z"

Let's see how is our dataset so far,

In [9]:
print(mission)

DataTree('None', parent=None)
│   Dimensions:  ()
│   Data variables:
│       *empty*
│   Attributes:
│       Conventions:  CF-1.9, ACDD-1.3, OG-1.0rc
│       title:        Spray Underwater glider demo
│       platform:     sp041
│       project:      California Underwater Glider Network
│       doi:          10.21238/S8SPRAY1618
│       comment:      Dataset for demonstration purposes only. Original dataset t...
└── DataTree('CTD')
        Dimensions:  (time: 5)
        Coordinates:
          * time     (time) datetime64[ns] 2019-12-16T17:22:41 ... 2019-12-16T17:23:13
        Data variables:
            press    (time) float64 5.32 4.36 3.4 2.32 1.56
            temp     (time) float64 15.51 15.51 15.51 15.51 15.51
            sal      (time) float64 33.56 33.56 33.56 33.56 33.56
        Attributes:
            long_name:         CTD sensor
            type:              CTD
            type_identifier:   https://vocab.nerc.ac.uk/collection/L05/current/130/
            maker:         

### GPS

Now it will start to get more interesting. We also have a GPS onboard, but it only works at the surface, before and after a dive. It's a completely different dimension of the CTD that can produce hundreds to thousands of values from a single dive. A flatten netCDF would require a lot of `FillValue` for GPS (while underwater) or another dimension. Instead of that, we will use another group for GPS with its own dimension.

Let's start with some values,

In [10]:
time_gps = np.array(['2019-12-16T14:41:00', '2019-12-16T17:30:03'], dtype='datetime64')
lat_gps = np.array([32.51754, 32.50146])
lon_gps = np.array([-119.80175, -119.82769])

And give some structure with `xarray` as we did before,

In [11]:
ds_gps = xr.Dataset(dict(
    time=(["time"], time_gps, {
        "long_name": "Time of each GPS fix",
        "standard_name": "time",
    }),
    lat=(["time"], lat_gps, {
        'long_name': 'Latitude of each GPS fix',
        'standard_name': 'latitude',
        'units': 'degrees_north',
    }),
    lon=(["time"], lon_gps, {
        'long_name': 'Longitude of each GPS fix',
        'standard_name': 'longitude',
        'units': 'degrees_east',
    }),
))

and keep time consistent, even not been required,

In [12]:
ds_gps["time"].encoding["calendar"] = "gregorian"
ds_gps["time"].encoding["units"] = "seconds since 1970-01-01T00:00:00Z"

and add to our mission

In [13]:
gps = DataTree(data=ds_gps, name="GPS", parent=mission)

gps.attrs["title"] = "GPS positions"
gps.attrs["comment"] = "GPS positioning is only possible when the glider is on the surface."

In [14]:
print(mission)

DataTree('None', parent=None)
│   Dimensions:  ()
│   Data variables:
│       *empty*
│   Attributes:
│       Conventions:  CF-1.9, ACDD-1.3, OG-1.0rc
│       title:        Spray Underwater glider demo
│       platform:     sp041
│       project:      California Underwater Glider Network
│       doi:          10.21238/S8SPRAY1618
│       comment:      Dataset for demonstration purposes only. Original dataset t...
├── DataTree('CTD')
│       Dimensions:  (time: 5)
│       Coordinates:
│         * time     (time) datetime64[ns] 2019-12-16T17:22:41 ... 2019-12-16T17:23:13
│       Data variables:
│           press    (time) float64 5.32 4.36 3.4 2.32 1.56
│           temp     (time) float64 15.51 15.51 15.51 15.51 15.51
│           sal      (time) float64 33.56 33.56 33.56 33.56 33.56
│       Attributes:
│           long_name:         CTD sensor
│           type:              CTD
│           type_identifier:   https://vocab.nerc.ac.uk/collection/L05/current/130/
│           maker:         

### Others

The same principle extends to other sensors that might operate in a different sampling frequency. For instance, the ADCP would require a few extra dimensions.

A convenient use of the lower level for this structure could be for a clean, QCed, and regularly binned dataset. While advanced users would probably prefer the high-resolution dataset, many users outside the glider community might prefer a basic dataset.

In [15]:
mission

DataTree('None', parent=None)
│   Dimensions:  ()
│   Data variables:
│       *empty*
│   Attributes:
│       Conventions:  CF-1.9, ACDD-1.3, OG-1.0rc
│       title:        Spray Underwater glider demo
│       platform:     sp041
│       project:      California Underwater Glider Network
│       doi:          10.21238/S8SPRAY1618
│       comment:      Dataset for demonstration purposes only. Original dataset t...
├── DataTree('CTD')
│       Dimensions:  (time: 5)
│       Coordinates:
│         * time     (time) datetime64[ns] 2019-12-16T17:22:41 ... 2019-12-16T17:23:13
│       Data variables:
│           press    (time) float64 5.32 4.36 3.4 2.32 1.56
│           temp     (time) float64 15.51 15.51 15.51 15.51 15.51
│           sal      (time) float64 33.56 33.56 33.56 33.56 33.56
│       Attributes:
│           long_name:         CTD sensor
│           type:              CTD
│           type_identifier:   https://vocab.nerc.ac.uk/collection/L05/current/130/
│           maker:             Sea-Bird Scientific
│           maker_identifier:  https://vocab.nerc.ac.uk/collection/L35/current/MAN0013/
│           model:             SBE 41CP CTD 
│           model_identifier:  https://vocab.nerc.ac.uk/collection/L22/current/TOOL0669/
│           serial_number:     75
│           calibration_date:  2019-10-22T00:00:00Z
└── DataTree('GPS')
        Dimensions:  (time: 2)
        Coordinates:
          * time     (time) datetime64[ns] 2019-12-16T14:41:00 2019-12-16T17:30:03
        Data variables:
            lat      (time) float64 32.52 32.5
            lon      (time) float64 -119.8 -119.8
        Attributes:
            title:    GPS positions
            comment:  GPS positioning is only possible when the glider is on the surf...

In [16]:
time = np.array(['2019-12-16T17:22:41', '2019-12-16T17:22:49', '2019-12-16T17:22:57', '2019-12-16T17:23:05', '2019-12-16T17:23:13'], dtype='datetime64')
pressure = np.array([5.32, 4.36, 3.4 , 2.32, 1.56])
temperature = np.array([15.511, 15.51 , 15.511, 15.512, 15.512])
salinity = np.array([33.561, 33.56 , 33.56 , 33.559, 33.56 ])


ds_binned = xr.Dataset(dict(
    time=(
        ["time"],
        np.array(['2019-12-16T17:22:49', '2019-12-16T17:23:09'], dtype='datetime64'),
        {
            "long_name": "Time",
            "standard_name": "time",
            "comment": "Derived from /CTD/time."
        }),
    press=(
        ["time"],
        np.array([5.00, 2.50]),
        {
        'long_name': 'Pressure in regular bins',
        'standard_name': 'sea_water_pressure',
        'units': 'dbar',
        'comment': 'Sea water pressure, equals 0 at sea-level. Derived from /CTD/pres.'}),
    temp=(
        ["time"],
        np.array([15.510666666666665, 15.512]),
        {
        'long_name': 'Sea Water Temperature',
        'standard_name': 'sea_water_temperature',
        'units': 'Celsius',
        'comment': 'Sea temperature in-situ ITS-90 scale. Derived from /CTD/temp.'}),
    sal=(
        ["time"],
        np.array([33.56033333333334, 33.5595]),
        {
        'long_name': 'Sea Water Salinity',
        'standard_name': 'sea_water_practical_salinity',
        'units': '1',
        'comment': 'Practical salinity computed using UNESCO 1983 algorithm. Derived from /CTD/sal.'}),
))

In [17]:
mission.to_netcdf("sp041_20191205T1757.nc")

# Data analysis



In [18]:
binned = mission["CTD"].ds.resample(time="15s").mean()

In [19]:
from datatree import open_datatree

open_datatree("sp041_20191205T1757.nc")

DataTree('None', parent=None)
│   Dimensions:  ()
│   Data variables:
│       *empty*
│   Attributes:
│       Conventions:  CF-1.9, ACDD-1.3, OG-1.0rc
│       title:        Spray Underwater glider demo
│       platform:     sp041
│       project:      California Underwater Glider Network
│       doi:          10.21238/S8SPRAY1618
│       comment:      Dataset for demonstration purposes only. Original dataset t...
├── DataTree('CTD')
│       Dimensions:  (time: 5)
│       Coordinates:
│         * time     (time) datetime64[ns] 2019-12-16T17:22:41 ... 2019-12-16T17:23:13
│       Data variables:
│           press    (time) float64 ...
│           temp     (time) float64 ...
│           sal      (time) float64 ...
│       Attributes:
│           long_name:         CTD sensor
│           type:              CTD
│           type_identifier:   https://vocab.nerc.ac.uk/collection/L05/current/130/
│           maker:             Sea-Bird Scientific
│           maker_identifier:  https://vocab.nerc.ac.uk/collection/L35/current/MAN0013/
│           model:             SBE 41CP CTD 
│           model_identifier:  https://vocab.nerc.ac.uk/collection/L22/current/TOOL0669/
│           serial_number:     75
│           calibration_date:  2019-10-22T00:00:00Z
└── DataTree('GPS')
        Dimensions:  (time: 2)
        Coordinates:
          * time     (time) datetime64[ns] 2019-12-16T14:41:00 2019-12-16T17:30:03
        Data variables:
            lat      (time) float64 ...
            lon      (time) float64 ...
        Attributes:
            title:    GPS positions
            comment:  GPS positioning is only possible when the glider is on the surf...